In [1]:
!pip install pillow

In [2]:
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

from keras.preprocessing.image import ImageDataGenerator

import PIL

Using TensorFlow backend.


Building the CNN
* Two ways of initializing the neural network, either as a sequence of layers or as a graph. 
* CNN is a sequence of layers
* Conv2D for images and Conv3D for videos (add time)
* Add the fully connected layers into a classic ANN
* The problem is non-linear so we need to have non-linearity in the model

In [3]:
# Initialising the CNN
classifier = Sequential()

# Step 1 - Convolution
# 32 = number of filters, start with 2^5=32, then 2^6=64, until 2^7=128
# (3,3) = (rows,columns)
# (64,64,3) = size of our input image
# relu = to avoid (-) values in the feature maps and have non-linearity in the CNN
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))

# Step 2 - Pooling
# pool_size = windows size of the feature map
# use less intensive computational power 
# recommendation size (2,2)
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Adding a second convolutional layer
# 1 CL train 85% and test 75% a difference of 10%
# improve the accuracy of 7% in the test dataset and reduce the diff in train/test
# a common practice is to double the number of feature detectors 32, 64, 128, ...
# 1 CL train 85% and test 82% a difference of 3%
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Step 3 - Flattening
# A huge vector where each node represent an input pixel image 
# Independent from all the pixels around it, only contains information from itself
# With a convolutional step we can codify spatial structure around the pixel
classifier.add(Flatten())

# Step 4 - Full connection
# Classic ANN composed of fully connected layers
# Convert the input image into one dimensional vector that contains information of the spatial structure
# Use the output (flatten huge vector) as an input of a ANN
# too-small to make the classifier a good model 
# too-big to make the classifier too highly compute-intensive
classifier.add(Dense(units = 128, activation = 'relu'))
# output layer
# sigmoid - to return probabilities of a binary outcome
classifier.add(Dense(units = 1, activation = 'sigmoid'))

# Compiling the CNN
# Stochastic gradient descent algorithm = adam
# Binary Cross Entropy corresponds to the logarithmic loss used in classification problem
# Since the output is binary, thus, a binary cross entropy loss function is needed
# Performance metric = accuracy
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

Fitting the CNN to the images
* Good performance on train dataset and poor performance on test dataset -> overfit
* Overfitting is also caused for few data train
* Can capture the patterns but fail at generalization
* More real data or data augmentation to produce synthetic data
* Rotate, flip, shift
* Shear mapping is a linear map that displaces each point in fixed direction, by an amount proportional to its signed distance from the line that is parallel to that direction and goes through the origin.


In [4]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)

* rescale the images to be between 0 and 1
* shear_range for some random transvections
* zoom changes
* horizontal flip
* batch_size: some random samples of our images will be included
* batch_size, contains the number of images that will go through the CNN after the weight will be updated

In [5]:
training_set = train_datagen.flow_from_directory('./dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 8000 images belonging to 2 classes.


In [6]:
test_set = test_datagen.flow_from_directory('./dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


How to improve accuracy?
* Add another convolutional layer
* Add another fully connected layer
* Increase accuracy and decrease loss
* Increase the size of the images

In [7]:
classifier.fit_generator(training_set,
                         steps_per_epoch = 8000,
                         epochs = 25,
                         validation_data = test_set,
                         validation_steps = 2000)

Epoch 1/25
8000/8000 [==============================] - 1034s 129ms/step - loss: 0.3754 - accuracy: 0.8227 - val_loss: 0.4232 - val_accuracy: 0.8110
Epoch 2/25
8000/8000 [==============================] - 960s 120ms/step - loss: 0.1546 - accuracy: 0.9384 - val_loss: 0.3419 - val_accuracy: 0.8194
Epoch 3/25
 568/8000 [=>............................] - ETA: 13:27 - loss: 0.0988 - accuracy: 0.9627

KeyboardInterrupt: 